## 环境准备

In [ ]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import proplot as pplt
pplt.rc.update({
    'fontsize':12,
    'font.family':'Arial',
    'edgefix':False,
    "svg.fonttype":'none',
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    })

from shapely.geometry import Polygon  # Geometry
from shapely import polygons  # Geometry creation

## 创建数据并调整到所需格式

提供的可以是1D的中心点坐标或者2D的中心点坐标，等间隔或者不等间隔均可  

In [ ]:
# [option1] 1D的中心点坐标
ds = xr.Dataset(
    coords=dict(  # 放在coords或者data_vars都可以
        latitude=(["y"], np.arange(32, 35, 0.5)+0.25),
        longitude=(["x"], np.arange(115, 120, 0.5)+0.25),
    ),
)

# 因为提供的是1D的经纬度坐标，所以需要进行广播
lat_1d = ds['latitude']
lon_1d = ds['longitude']
ds['lat_2d'], ds['lon_2d'] = xr.broadcast(lat_1d, lon_1d)

In [ ]:
# [option2] 2D的中心点坐标且不规则
# 实则相当于是一张经纬度查找表
# 规则但不等间隔的话与option1完全相同
# 不规则的话ds其实无法给出1D的经纬度坐标

# 先假设是规则的，这里使用meshgrid方式来创建
lat_1d = np.arange(32, 35, 0.5)+0.25
lon_1d = np.arange(115, 120, 0.5)+0.25
lon_2d, lat_2d = np.meshgrid(lon_1d, lat_1d, indexing='xy')

# 然后增加随机性
# 计算距离某个中心点的权重来生成不规则网格
distance = np.sqrt((lon_2d - 117.5) ** 2 + (lat_2d - 33.5) ** 2)
lon_2d += np.exp(-distance**2/2)*0.25
lat_2d += np.exp(-distance**2/2)*0.25

# 创建dataset
ds = xr.Dataset({
    'lat_2d': (('y', 'x'), lat_2d),
    'lon_2d': (('y', 'x'), lon_2d)
})

In [ ]:
# [check]
# 因为纯属自己构造，可在过程中看下是不是心目中假设的数据形式
plt.scatter(lon_2d, lat_2d, s=10, color='blue', alpha=0.5)
plt.xlim(115, 120)
plt.ylim(32, 35)
plt.title('Irregular Grid with Center Density')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# 对纬度进行降序排列使实际矩阵与地图视角一致
# 1D的'latitude'只适用于创建的规则四边形网格
# 这里创建虚拟字段来提高代码的适配性
ds.coords['latitude_vir'] = ds['lat_2d'].median(dim='x')
ds = ds.sortby('latitude_vir', ascending=False)

### 绘制2D经纬度数据的可视化

In [ ]:
# [check] 
# 注意这里x,y只是单纯的索引，可以理解为就是单纯的矩阵的可视化

# 如果直接用plot会发现坐标轴是整数索引值，纬度坐标是逆序的
ds['lat_2d'].plot(); plt.show()
ds['lon_2d'].plot(); plt.show()

# 如果用imshow则与地图视角一致
plt.imshow(ds['lat_2d'].values);plt.colorbar(); plt.show()
plt.imshow(ds['lon_2d'].values);plt.colorbar(); plt.show()

# 当然也可以自己控制坐标轴顺序来与地图视角一致
ds['lat_2d'].plot(yincrease=False); plt.show()
plt.imshow(ds['lat_2d'].values, origin='upper');plt.colorbar(); plt.show()

In [ ]:
# 综上两种绘制结果如下
fig, axs = plt.subplots(2, 2, figsize=(10, 6))

# 使用xarray的plot
ds["lat_2d"].plot(ax=axs[0, 0], yincrease=False)
ds["lon_2d"].plot(ax=axs[0, 1], yincrease=False)

# 使用matplotlib的imshow
axs[1, 0].imshow(ds["lat_2d"].values, origin="upper")
axs[1, 0].set_title("Latitude (2D) - imshow")
plt.colorbar(axs[1, 0].images[0], ax=axs[1, 0])
axs[1, 1].imshow(ds["lon_2d"].values, origin="upper")
axs[1, 1].set_title("Longitude (2D) - imshow")
plt.colorbar(axs[1, 1].images[0], ax=axs[1, 1])

plt.tight_layout()
plt.show()

In [ ]:
# 绘制到真实经纬度地图上
fig, axs = pplt.subplots(ncols=2, nrows=1, proj='pcarree', figsize=(10, 3))

axs[0].pcolormesh(ds['lon_2d'], ds['lat_2d'], ds['lon_2d'], cmap='jet', edgecolors='k', linewidth=0.1)
axs[1].pcolormesh(ds['lon_2d'], ds['lat_2d'], ds['lat_2d'], cmap='jet', edgecolors='k', linewidth=0.1)

axs[0].scatter(ds['lon_2d'], ds['lat_2d'], c='k', s=1)
axs[1].scatter(ds['lon_2d'], ds['lat_2d'], c='k', s=1)

axs.format(
        latlim=(31.5, 35.5), 
        lonlim=(114.5, 120.5),
        latlabels='l',
        lonlabels='b',
        lonlines=0.5,
        latlines=0.5,
        )

## 将插值后的坐标点分配为网格的形状参数

### 获得密集网格下的经纬度坐标

In [ ]:
# 合并并排序网格中心点索引和边界点索引
# 为了计算每个网格的四角和四边中心点坐标，在每个原始格点之间插入一个新的格点
y_indices = np.linspace(-0.5, ds.y.values[-1]+0.5, 2*len(ds.y)+1)
x_indices = np.linspace(-0.5, ds.x.values[-1]+0.5, 2*len(ds.x)+1)

# 插值获得索引位置的实际经纬度
# 使用xarray的interp方法进行插值，获取网格边界的经纬度坐标，可能是这里最大的技巧！
# interp方法会对原始数据集经纬度进行插值，插值的位置由y_indices和x_indices指定。
# kwargs={"fill_value": "extrapolate"} 表示使用外插法填充超出原始数据范围的值。
ds_interp = ds.interp(y=y_indices, x=x_indices, 
                      method="cubic",
                      kwargs={"fill_value": "extrapolate"}
                      )

# 保持x和y为coords变量
ds_interp = ds_interp.set_coords(["x", "y"])

In [ ]:
# [check]
ds_interp

### 将ds索引映射到ds_interp索引获取边界点坐标构建多边形

现在，ds_interp 数据集包含了每个网格的四个角和四边中心点的经纬度坐标。

第1个ds网格(i=0, j=0)的构成坐标：
- 左上角: ds_interp.isel(y=0, x=0)
- 上边中: ds_interp.isel(y=0, x=1)
- 右上角: ds_interp.isel(y=0, x=2)
- 右边中: ds_interp.isel(y=1, x=2)
- 右下角: ds_interp.isel(y=2, x=2)
- 下边中: ds_interp.isel(y=2, x=1)
- 左下角: ds_interp.isel(y=2, x=0)
- 左边中: ds_interp.isel(y=1, x=0)

In [ ]:
# 构建原先的ds每个网格自身的索引
i, j = np.indices((len(ds.y), len(ds.x)))  # i和j都是(ny, nx)大小

# 映射到ds_interp的索引值并组合为(ny, nx, 8)大小的数组
poly_y = np.stack([2*i, 2*i, 2*i, 2*i+1, 2*(i+1), 2*(i+1), 2*(i+1), 2*i+1], axis=-1)
poly_x = np.stack([2*j, 2*j+1, 2*(j+1), 2*(j+1), 2*(j+1), 2*j+1, 2*j, 2*j], axis=-1)

# 组合每个位置(ny, nx)的poly_y和poly_x形成索引从而得到每个位置这8个点的经/纬度
# 对于每一对 (poly_y[i, j, k], poly_x[i, j, k])，提取 lat_interp 中的相应值
# 因为是两个索引数组（poly_y 和 poly_x），而 lat_interp 也是二维(2*ny+1, 2*nx+1)
# 所以它们会被逐元素匹配，将其两个索引数组相同位置的每组值作为二维数组的行列索引
# 索引完成后，结果数组的形状等同于索引数组的形状 (ny, nx, 8)
poly_lat = ds_interp["lat_2d"].values[poly_y, poly_x]
poly_lon = ds_interp["lon_2d"].values[poly_y, poly_x]

# 沿最后一个轴堆叠这两个 (ny, nx, 8) 形成(ny, nx, 8, 2) 的经纬度数组
poly_coords = np.stack([poly_lon, poly_lat], axis=-1)

# 借助polygons直接批量构建多边形，它可以接受一个形状为 (..., N, 2) 的数组，
# 为每个 (ny, nx) 位置创建一个多边形，并返回一个形状为 (ny, nx) 的多边形数组
poly_grid = polygons(poly_coords)

In [ ]:
# [check]
print(poly_grid.shape)
poly_grid

#### 详细过程1: 使用循环遍历获取网格构成坐标

In [ ]:
# [check]
# 创建存储网格信息的空列表
poly_grid = []

# 遍历原始ds格点
for i in range(len(ds.y)):
    for j in range(len(ds.x)):

        # 获取ds_interp网格中心点索引
        center_y = 2*i + 1
        center_x = 2*j + 1

        # 获取当前网格的四个角点索引(左上、右上、右下、左下)
        corner_y = [2*i, 2*i, 2*(i+1), 2*(i+1)]
        corner_x = [2*j, 2*(j+1), 2*(j+1), 2*j]

        # 获取边界中点索引(上、右、下、左)
        edge_y = [2*i, 2*i+1, 2*(i+1), 2*i+1]
        edge_x = [2*j+1, 2*(j+1), 2*j+1, 2*j]

        # 将角点索引 corner_y 和边界点索引 edge_y 交错组合
        # stack 且 axis=1 会将两个数组按列堆叠，形成一个2列的数组
        # 每行的第1列是 corner_y 中的值，第2列是 edge_y 中的值
        # 然后展平成(左上角、上边中、右上角、右边中、右下角、下边中、左下角、左边中)
        poly_y = np.stack([corner_y, edge_y], axis=1).flatten()
        poly_x = np.stack([corner_x, edge_x], axis=1).flatten()
        
        # 获取这8个位置的实际坐标
        # 注意这里需要使用 values 来取值，否则得到的是交错的坐标即8*8的矩阵
        poly_lat = ds_interp['lat_2d'].values[poly_y, poly_x]
        poly_lon = ds_interp['lon_2d'].values[poly_y, poly_x]

        # 使用Polygon类来创建每一个多边形对象
        # zip函数将 poly_lon 和 poly_lat 中的元素配对，形成一系列的(经度,纬度)元组
        # Polygon类接受这些元组并创建一个多边形对象
        polygon = Polygon(zip(poly_lon, poly_lat))

        # 将每个网格的存储在 poly_grid 列表中
        # 如需获取坐标可以使用下面的方式
        # coords_list = list(polygon.exterior.coords)  # 返回(lon, lat)对的列表
        # xy_tuple = polygon.exterior.coords.xy  # 返回 (x数组, y数组)
        poly_grid.append(polygon)

# 还原为ds的维度
poly_grid = np.array(poly_grid).reshape(len(ds.y), len(ds.x))

#### 详细过程2: 直接使用数组索引机制加速构建网格坐标

In [ ]:
# [check]
# 构建ds每个网格的索引
ny, nx = len(ds.y), len(ds.x)
i, j = np.indices((ny, nx))  # i和j都是(ny, nx)大小

# 直接把 corner 和 edge 交叉组合为 (8, ny, nx) 大小的数组
# 当中每一个值是当前网格角点或边的中点映射到 ds_interp 的索引值
poly_y = np.array([2*i, 2*i, 2*i, 2*i+1, 2*(i+1), 2*(i+1), 2*(i+1), 2*i+1])
poly_x = np.array([2*j, 2*j+1, 2*(j+1), 2*(j+1), 2*(j+1), 2*j+1, 2*j, 2*j])

# 获取插值后的经纬度数组
lat_interp = ds_interp["lat_2d"].values  # (2*ny+1, 2*nx+1)
lon_interp = ds_interp["lon_2d"].values  # (2*ny+1, 2*nx+1)

# 注意上面ny和nx针对的是ds，仅只指示每一个需要处理的网格
# 而从ds_interp获得8个点的经/纬度是要组合每个位置的poly_y和poly_x形成索引
# 对于ds中的单个位置，根据8个点的索引，取出对应ds_interp的数值
tmp_lat = lat_interp[poly_y[:, 5, 7], poly_x[:, 5, 7]]
tmp_lon = lon_interp[poly_y[:, 5, 7], poly_x[:, 5, 7]]
print(tmp_lat, tmp_lon)

# 直接批量取值，注意这里的索引机制
# 首先检查索引数组的形状是否一致（poly_y 和 poly_x 均为 (8, ny, nx)，一致），将其作为输出数组的形状
# 对于每一对 (poly_y[i, j, k], poly_x[i, j, k])，提取 lat_interp 和 lon_interp 中的相应值
# 因为是两个索引数组（poly_y 和 poly_x），而 lat_interp 和 lon_interp 也是二维
# 所以它们会被逐元素匹配，将其两个索引数组相同位置的每组值作为二维数组的行列索引
# 索引完成后，结果数组的形状等同于索引数组的形状 (8, ny, nx)。
poly_lat = lat_interp[poly_y, poly_x]
poly_lon = lon_interp[poly_y, poly_x]



# # 下面的线性索引方案反而更复杂且需要反复变换数组维度
# # 将索引展平以方便后续批量化的取值
# flat_poly_y = poly_y.reshape(8, -1)  # (8, ny*nx)
# flat_poly_x = poly_x.reshape(8, -1)  # (8, ny*nx)

# # 转化为线性索引
# # 原来需要两个索引的组合，现在只需要一个，尺寸仍然为 (8, ny*nx)
# flat_indices = np.ravel_multi_index((flat_poly_y, flat_poly_x), (2*ny+1, 2*nx+1))
# # 可以理解成 (行号, 列号) -> 行号*待取值数组每行的列数+列号
# print(flat_poly_y[:, 35], flat_poly_x[:, 35], flat_indices[:, 35])
# print(flat_poly_y[:, 35]*(2*nx+1)+flat_poly_x[:, 35])

# # 利用线性索引在展平的数据上提取对应位置的数据
# flat_lat_values = lat_interp.ravel()[flat_indices]  # (8, ny*nx)
# flat_lon_values = lon_interp.ravel()[flat_indices]  # (8, ny*nx)

# # 还原形状为 (8, ny, nx) 的纬度和经度值
# poly_lat = flat_lat_values.reshape(8, ny, nx)
# poly_lon = flat_lon_values.reshape(8, ny, nx)



# 跟上面单个位置的结果检查核对，也可以跟过程1的结果核对
print(poly_lat[:, 5, 7], poly_lon[:, 5, 7])

# 组合经纬度数组，首先沿最后一个轴堆叠这两个 (8, ny, nx)
# 形成形状为 (8, ny, nx, 2) 的数组，然后调整轴顺序使其变为 (ny, nx, 8, 2)
poly_coords = np.stack([poly_lon, poly_lat], axis=-1).transpose(1, 2, 0, 3)
print(poly_coords.shape)

# 借助shapely库的polygons直接批量构建多边形
# 它可以接受一个形状为 (..., N, 2) 的数组，为每个 (ny, nx) 位置创建一个多边形，并返回一个形状为 (ny, nx) 的多边形数组
poly_grid = polygons(poly_coords)
print(poly_grid.shape)

## 在地图上可视化这些创建的多边形

In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.graph_objects as go

用 geopandas 直接加载这些多边形

In [ ]:
gdf = gpd.GeoDataFrame(poly_grid.ravel(), columns=['geometry'])
gdf.plot()  # 绘制出图的方式
gdf.explore()  # 交互式可视化

或者使用 plotly 来绘制

In [ ]:
# 将 GeoDataFrame 转换为 Pandas DataFrame 并提取坐标
df = pd.DataFrame(gdf)
df['lon'] = df['geometry'].apply(lambda geom: [coord[0] for coord in geom.exterior.coords])
df['lat'] = df['geometry'].apply(lambda geom: [coord[1] for coord in geom.exterior.coords])

# 等于以下代码，注意返回的是包含x和y的元组，并且x和y都是array.array，还要转为列表
# df['lon'] = df['geometry'].apply(lambda geom: geom.exterior.coords.xy[0].tolist())
# df['lat'] = df['geometry'].apply(lambda geom: geom.exterior.coords.xy[1].tolist())

# 创建 Plotly 地图
fig = go.Figure(go.Scattergeo())
for index, row in df.iterrows():
    fig.add_trace(go.Scattergeo(
        lon=row['lon'],
        lat=row['lat'],
        text=str(list(row['geometry'].exterior.coords)),  # 显示坐标信息
        hoverinfo='text',
        mode='lines',
        # name=row['name']
    ))

fig.update_layout(
    geo=dict(
        showland=True,
        showlakes=True,
        landcolor='rgb(212, 212, 212)',
        lakecolor='rgb(255, 255, 255)',
        showocean=True,
        oceancolor='rgb(191, 221, 255)'
    ),
    title='Polygons with Coordinates'
)
fig.show()